In [ ]:
import pdfplumber

with pdfplumber.open(r"C:\Users\bbanz\OneDrive\Documents\notebook\notebook\хичээл 3A\Мэдээлэл хайлт\biydaalt\mnt_en.pdf") as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

with open("mnt_en_text.txt", "w", encoding="utf-8") as f:
    f.write(text)

In [ ]:
import re
import json
import pdfplumber

with pdfplumber.open(r"C:\Users\bbanz\OneDrive\Documents\notebook\notebook\хичээл 3A\Мэдээлэл хайлт\biydaalt\mnt_en.pdf") as pdf:
    for page in pdf.pages:
        text += page.extract_text() + "\n"

with open("mnt_en_text.txt", "w", encoding="utf-8") as f:
    f.write(text)
with open("mnt_en_text.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
lines = raw_text.strip().split('\n')
chapter_title = ""
content_start = 0
for i, line in enumerate(lines):
    stripped = line.strip()
    if stripped and not re.match(r'^\d+\.', stripped):
        content_start = i + 1
        break
content = '\n'.join(lines[content_start:]).strip()
section_pattern = r'\n\s*(\d+)\.\s+'
sections = re.split(section_pattern, '\n' + content)  # prepend \n to handle first section
data = []
chapter_title ="The Secret History of The Mongol"
for i in range(1, len(sections), 2):
    if i + 1 >= len(sections):
        break
    section_number = int(sections[i].strip())
    section_text = sections[i + 1].strip()
    section_text = re.sub(r'\n+', ' ', section_text).strip()
    entry = {
        "chapter": chapter_title,
        "section": section_number,
        "text": section_text
    }
    data.append(entry)
with open("mnt_clean.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)
print(f"Гүйцэтгэл амжилттай! {len(data)} зүйлтэй mnt_clean.json файл үүссэн.")

In [ ]:
import re, json, os, html
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
import ipywidgets as widgets

def translate_mn_en(text):
    t = text.lower()
    replacements = {
        r"\bчингис хаан\b": "genghis khan",
        r"\bтэмүүжин\b": "temujin",
        r"\bбөртэ\b": "borte",
        r"\bхэзээ төрсөн бэ\b": "born when",
        r"\bхэн бэ\b": "who",
        r"\bюу болсон бэ\b": "what happened",
        r"\bбулаасан\b": "kidnapped",
        r"\bхаанаас\b": "from where",
        r"\bяаж\b": "how",
        r"\bгэрлэсэн\b": "married",
        r"\bхэдтэй\b": "how old"
    }
    for pattern, repl in replacements.items():
        t = re.sub(pattern, repl, t, flags=re.IGNORECASE
    return t
def translate_en_mn(text):
    return text

class MNT_SearchEngine:

    def __init__(self, file_path_en="mnt_clean.json"):

        self.stop_words = {"a","an","the","and","is","of","in","to","was","were","by","as","with","from","at","on","he","she","it","they","but","for","this","that","be","have","had"}
        self.documents = self._load_data(file_path_en)

        if self.documents:
            self._initialize_index()
            print(f"✅ Successfully loaded: {len(self.documents)} sentences")
        else:
            print("❌ Index creation failed.")

    def _load_data(self, file_path):
        try:
            actual_path = file_path
            if not os.path.exists(file_path) and os.path.exists(os.path.basename(file_path)):
                actual_path = os.path.basename(file_path)
            with open(actual_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            docs = []

            for i, item in enumerate(data):
                text = item.get("text","").strip()
                section = item.get("section", i+1)
                docs.append({"doc_id": f"MNT_S{section:02d}", "text": text, "section": section, "index": i})
            return docs
        except Exception as e:
            sample_texts = [

                "Börte-chino, a descendant of Genghis Khan and born from heaven, crossed the sea with his wife Gua-Maral.",
                "The wise Dobun Mergen married Alan Gua.",
                "Alan Gua gave birth to two sons after Dobun Mergen's death.",
                "Temujin was born holding a blood clot in his hand.",
                "In 1206 Temujin was proclaimed Chinggis Khaan."

            ]
            docs = []
            for i, txt in enumerate(sample_texts):
                sec = i+1
                docs.append({"doc_id": f"MNT_S{sec:02d}", "text": txt, "section": sec, "index": i})
            return docs
    def en_tokenize(self, text):
        if not isinstance(text, str): return []
        tokens = re.findall(r"\b[a-zA-Z]+\b", text.lower())
        tokens = [t for t in tokens if t not in self.stop_words and len(t)>2]
        return tokens

    def _initialize_index(self):
        tokenized = [self.en_tokenize(d["text"]) for d in self.documents]
        self.bm25 = BM25Okapi(tokenized)
        self.vectorizer = TfidfVectorizer(tokenizer=self.en_tokenize, lowercase=True)
        doc_strings = [" ".join(tokens) for tokens in tokenized]
        self.tfidf_matrix = self.vectorizer.fit_transform(doc_strings)

    def bm25_search_en(self, query_en, k=5):
        tokens = self.en_tokenize(query_en)
        if not tokens: return []
        raw = self.bm25.get_scores(tokens)
        mn, mx = float(np.min(raw)), float(np.max(raw))
        if mx-mn>1e-9:
            norm = (raw-mn)/(mx-mn)
        else:
            norm = raw
        top_idx = np.argsort(norm)[::-1][:k]
        return [(self.documents[i]["doc_id"], float(norm[i]), self.documents[i]["text"], i) for i in top_idx if norm[i]>0]
    def get_best_sentence_en(self, query_mn):
        q_en = translate_mn_en(query_mn)
        res = self.bm25_search_en(q_en, k=1)
        if res:
            doc_id, score, text_en, idx = res[0]

            best_sentence_en = text_en
            response = (
                f"<div style='background:#f0f8ff;padding:15px;border-radius:8px;border-left:5px solid #0066cc'>"
                f"<b>Best Matching Sentence:</b><br>"
                f"<p style='font-size:1.1em;'>{best_sentence_en}</p>"
                f"<i><small>Source: {doc_id} (Score: {score:.3f})</small></i>"
                f"</div>"
            )
            return response

        return "Sorry, no relevant results were found for your query."
    
    def chatbot_response(self, query_mn):

        return self.get_best_sentence_en(query_mn)

search_engine = MNT_SearchEngine(file_path_en="mnt_clean.json")

output = widgets.Output(layout=widgets.Layout(height='400px', overflow_y='auto', border='1px solid #ccc'))
with output:

    display(HTML("""

    <div style='background:#f0f8ff;padding:20px;border-radius:10px;text-align:center'>

        <h3>Hello!</h3>

        <p>Ask a question about The Secret History of the Mongols (in Mongolian).</p>

        <p>The system will search using the English translation and return the best English sentence.</p>
        <p>Жишээ: <i>Чингис хаан хэзээ төрсөн бэ?</i></p>
        <p><i>Бөртэ хэн булаасан бэ?</i></p>
        <p><i>1206 онд юу болсон бэ?</i></p

    </div>

    """))

def show_best_result(b=None):
    query_mn = query_input.value.strip()
    if not query_mn:

        return
    q_en = translate_mn_en(query_mn)
    with output:
        output.clear_output()
        display(HTML(f"<h3>Query: <span style='color:#0066cc'>{html.escape(query_mn)}</span></h3>"))
        display(HTML(f"<small><i>Search Term (EN): {html.escape(q_en)}</i></small><hr>"))
        display(HTML(search_engine.chatbot_response(query_mn)))

query_input = widgets.Text(
    placeholder="Example: Чингис хаан хэзээ төрсөн бэ?",
    description="Query:",
    layout=widgets.Layout(width="80%")

)
btn = widgets.Button(description="Search & Answer (EN)", button_style="success")
btn.on_click(show_best_result)
query_input.on_submit(show_best_result)
display(widgets.VBox([widgets.HBox([query_input, btn]), output]))

✅ Successfully loaded: 277 sentences


C:\Users\bbanz\AppData\Local\Temp\ipykernel_16136\2071375793.py:317: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  query_input.on_submit(show_best_result)
